In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.dates as mdates
from datetime import datetime, time ,date ,timedelta
from dateutil.relativedelta import relativedelta
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

from datetime import date
today = date.today().strftime("%Y%m%d")

import warnings
warnings.filterwarnings("ignore")

In [2]:
def flatten (table):
    if type(table.columns)==pd.MultiIndex:
        columns_to_look = [name_tmp for name_tmp in table.columns]

        columns_df = [ str(t[0])+'_'+str(t[1]) for t in columns_to_look]
        columns_df.insert(0,table.index.name)

        df = pd.DataFrame(columns = columns_df)

        index = 0
        for i in table.index:
            row = [table[r][i] for r in columns_to_look]
            row.insert(0,i)
            df.loc[index] = row
            index = index + 1
        return(df)
    else :
        table = pd.DataFrame(table)
        table.reset_index(level=0, inplace=True)
        return table

In [2]:
repertoire = 'C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/SEGSTRAT'

# SEG_STRAT to csv

Create a csv with the field IDIND - NUMCDE - DATCDE - SEGMENT

In [6]:
def remove_zero(x):
    if x[-2] == '0':
        return x[:-2]+x[-1]
    else:
        return x

In [40]:
df = pd.DataFrame()
for year in range(2017,2022):
    print(year)
    gc.collect()
    df_tmp = pd.read_csv('./Vertbaudet/France/src/Original/Lignes_commandes/Export_FR_LIGNECOMMANDE_'+str(year)+'.csv', 
                              encoding = 'unicode_escape', 
                              usecols = ['IDIND', 'NUMCDE', 'DATCDE'])
    
    df_tmp = df_tmp.replace({'$null$': np.nan})
    df_tmp['DATCDE'] = pd.to_datetime(df_tmp['DATCDE'])
    df_tmp['IDIND'] = df_tmp['IDIND'].astype(float)
    
    ### Load segments
    repertoire = 'C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/SEGSTRAT/'+str(year)
    df_seg = pd.DataFrame()
    for file in os.listdir(repertoire):
        file_name_tmp = os.path.join(repertoire,file)
        df_seg_tmp = pd.read_csv(file_name_tmp,sep=",",encoding= "unicode_escape")
        df_seg = pd.concat([df_seg,df_seg_tmp])
        del(df_seg_tmp)
    
    ### Format segments  
    df_seg = df_seg[df_seg['PERIODE_MENS'] != '$null$']
    df_seg.PERIODE_MENS = df_seg.PERIODE_MENS.astype(int)
    df_seg.PERIODE_MENS = df_seg.PERIODE_MENS.astype(str)
    df_seg = df_seg.drop_duplicates(subset = ['IDIND', 'PERIODE_MENS'])
    df_seg = df_seg.groupby(['IDIND', 'PERIODE_MENS']).agg({'SEG_STRAT': 'first'}).unstack()
    df_seg = df_seg.droplevel(level = 0, axis = 1)
    
    if '$null$' in df_seg.columns:
        df_seg = df_seg.drop(columns = '$null$')
    gc.collect()
    
    df_seg.columns = list(map(remove_zero,df_seg.columns))
    if '$null$' in df_seg.index:
        df_seg = df_seg.drop(index = '$null$')
    df_seg_index = df_seg.index.astype(float)
    
    
    ### Compute segment column
    list_month = list(map(lambda x : int(x[4:]), df_seg.columns))

    def map_seg(x):
        if x[0] in df_seg.index:
            for month in list_month:
                if x[1] <= date(year, month, 28):
                    return df_seg[str(year)+str(month)][x[0]]
                    break
            return np.nan
        else:
            return np.nan
        
    df_tmp['SEGMENT'] = df_tmp[['IDIND', 'DATCDE']].apply(map_seg, axis = 1)
    
    df = pd.concat([df,df_tmp])
  
    del df_tmp
    gc.collect()

2017
2018
2019
2020
2021


OSError: [Errno 22] Invalid argument

In [45]:
df.to_csv('./Vertbaudet/France/src/SEGFR.CSV')

# Import df

In [3]:
df_articles = pd.read_csv('./Vertbaudet/France/src/Original/Articles/Export_FR_Articles_Julie.csv',
                                     usecols = ["DEPARTEMENT_NIV2","CODSAI","REFSTK", "TOP_MN_Max"])

df_codsoc = pd.read_csv('./Vertbaudet/France/src/20210921_CODSOC.csv')

df_client_dept = pd.read_csv('./20210923_Export_tab_FR/20210921_INDIVIDUS.tab', sep = '\t',
                 usecols = ['IDIND', 'CODPOST', "DATPREMCDEWEB"])

clients_mag = pd.read_csv("./Vertbaudet/France/src/20211004_UNIQUE_IDIND_TICKET.csv",sep=",")

df_client = pd.read_csv('./Vertbaudet/France/Output_old/iris_med_rev_clients.csv', usecols = ['IDIND', 'Mediane'])

df_IRIS_by_dep = pd.read_excel('./Vertbaudet/France/INSEE_data/INSEE_IRIS_by_dep.xlsx', usecols = ['CODGEO','MED18'])

In [4]:
gc.collect()

267

In [5]:
gc.collect()
df = pd.DataFrame()
for year in range(2017,2022):
    print(year)
    gc.collect()
    df_tmp = pd.read_csv('./Vertbaudet/France/src/Original/Lignes_commandes/Export_FR_LIGNECOMMANDE_'+str(year)+'.csv', 
                              encoding = 'unicode_escape', 
                              usecols = ['IDIND', 'IDCLI', 'IDFOYER', 'DATCDE', 'DN', 'DBI', 'QTE', 'NUMCDE', 'CODSAI','REFSTK'])
    
    ### df_evt_tclub
    df_evt_tclub = pd.read_csv('C:/Users/pierrick/eleven/Engagements - Vertbaudet/3. Data received/3. Extract/1. France/20211022_TYPEVT_'+str(year)[2:]+'.csv')
    df_tmp = df_tmp.merge(df_evt_tclub, how = 'left', on = ['IDCLI', 'DATCDE', 'REFSTK', 'CODSAI', 'NUMCDE'])
    del df_evt_tclub
    gc.collect()
    
    
    ### Merge with article table to get DEPARTEMENT_NIV2
    df_tmp = df_tmp.merge(df_articles, how = 'left', on = ["CODSAI","REFSTK"])
    
    ### Replace $null$
    df_tmp = df_tmp.replace({'$null$': np.nan})
    ### Clean IDIND index 
    df_tmp.IDIND = df_tmp.IDIND.astype(float)
    df_tmp.IDFOYER = df_tmp.IDFOYER.astype(float)
    
    ### Filter mag clients
    df_tmp = df_tmp[~df_tmp.IDIND.isin(clients_mag.IDIND)]

    ### Filter non-french command with CODSOC
    df_tmp = df_tmp.merge(df_codsoc, how = 'left', on = 'NUMCDE')
    df_tmp = df_tmp[df_tmp.CODSOC==0]

    #### Merge to get Mediane rev
    df_tmp = df_tmp.merge(df_client, how = 'left', on = 'IDIND')

    ### add departments and cohort
    df_tmp = df_tmp.merge(df_client_dept.replace({'$null$': np.nan}), how = 'left', on = 'IDIND')
    df_tmp = df_tmp[df_tmp.DATPREMCDEWEB != '$null$']
    df_tmp['DATCDE'] = pd.to_datetime(df_tmp["DATCDE"])
    df_tmp['DATPREMCDEWEB'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"])
    #df_tmp['DEPARTEMENT'] = df_tmp['CODPOST'].apply(lambda x : str(x)[:2])
    df_tmp['COHORT'] = pd.to_datetime(df_tmp["DATPREMCDEWEB"]).dt.year
    
    ### filter cohort
    df_tmp = df_tmp[(df_tmp['COHORT'] >= 2017) & (df_tmp['COHORT'] < 2021)]
    
    ### filter on DN <= DB
    df_tmp = df_tmp[df_tmp.DN <= df_tmp.DBI]
    df_tmp = df_tmp[df_tmp.DN >= 0]

    df = pd.concat([df,df_tmp[['IDIND', 'DN', 'NUMCDE', 'DATCDE', 'DATPREMCDEWEB', 'DEPARTEMENT_NIV2', 'COHORT']]])
  
    del df_tmp
    gc.collect()

2017
2018
2019
2020
2021


In [7]:
### delete useless dataframes
for x in [df_articles, df_codsoc, df_client_dept, clients_mag, df_client, df_IRIS_by_dep]:
    del x

In [9]:
# Exclude B2B clients
df_b2b = pd.read_csv('./Vertbaudet/France/src/20211012_TB2B.csv')
df_b2b[['IDIND', 'TB2B']] = df_b2b[['IDIND', 'TB2B']].astype(float)
df = df.merge(df_b2b, how = 'left', on = 'IDIND')
df = df[df['TB2B']==0]
df = df.drop(columns = ['TB2B'])
del df_b2b

In [88]:
### Add ORDER_NUMBER
nb_order =  flatten(df.sort_values(by='DATCDE',ascending=True).groupby(['NUMCDE']).agg({'IDIND':'first', 'DATCDE': 'first'}))
nb_order['ORDER_NUMBER'] = nb_order.sort_values(by='DATCDE',ascending=True).groupby(['IDIND']).cumcount()+1
df = pd.merge(df,nb_order[['NUMCDE','ORDER_NUMBER']],on='NUMCDE',how='left')

### Client Category

In [10]:
def customer_category_attribution(x):
    if x==1:
        return 'One-timer'
    if x==2:
        return 'Two-timer'
    else:
        return 'Recurring'

### Add CLIENT_CATEGORY
nb_commande = flatten(df.groupby(['IDIND']).agg({'NUMCDE':pd.Series.nunique}))
nb_commande.columns = ['IDIND','NB_ORDERS']
df = pd.merge(df,nb_commande,on='IDIND',how='left',suffixes=(False,False))
df['CLIENT_CATEGORY'] = df.NB_ORDERS.apply(lambda x: customer_category_attribution(x))

### LTV

In [11]:
import datetime
from datetime import datetime, time ,date ,timedelta

# Add DAT_12 and DAT_14 column
df['DAT_12'] = df['DATPREMCDEWEB'] + timedelta(days=365)
df['DAT_24'] = df['DATPREMCDEWEB'] + timedelta(days=730)

### Add LTV_12 column
df_ltv_12 = df[df['DATCDE']<=df['DAT_12']][['IDIND', 'DN']]
df_ltv_12 = df_ltv_12.groupby('IDIND').agg({'DN': sum})
df_ltv_12 = df_ltv_12.rename(columns = {'DN': 'LTV_12'})
print(df_ltv_12['LTV_12'].mean())
df = df.merge(df_ltv_12, how = 'left', on = 'IDIND')

del df_ltv_12

### Add LTV_24 column
df_ltv_24 = df[df['DATCDE']<=df['DAT_24']][['IDIND', 'DN']]
df_ltv_24 = df_ltv_24.groupby('IDIND').agg({'DN': sum})
df_ltv_24 = df_ltv_24.rename(columns = {'DN': 'LTV_24'})
print(df_ltv_24['LTV_24'].mean())
df = df.merge(df_ltv_24, how = 'left', on = 'IDIND')

del df_ltv_24

gc.collect()

121.9110836696581
150.33917873961332


0

### Churn_delay

In [14]:
### Add churn_delay field
df_churn = df.sort_values(by = 'DATCDE', ascending = True).groupby('IDIND').agg({'DATCDE': 'last', 'DATPREMCDEWEB': 'last', 'IDIND':'last'})
df_second = df[df['ORDER_NUMBER'] == 2].drop_duplicates('IDIND')[['IDIND', 'DATCDE']].rename(columns = {'DATCDE': 'DATCDE_2'})
df_churn = df_churn.merge(df_second, how = 'left', left_index = True,  right_on = 'IDIND')
df_churn = df_churn.reset_index()
df_churn['CHURN_DELAY'] = (df_churn['DATCDE'] + timedelta(days=365)) - df_churn['DATPREMCDEWEB']
df_churn['CHURN_DELAY_2'] = (df_churn['DATCDE'] + timedelta(days=365)) - df_churn['DATCDE_2']
df_churn = df_churn.drop(columns = 'index')

df = df.merge(df_churn[['IDIND','CHURN_DELAY_2', 'CHURN_DELAY']], how = 'left', left_on = 'IDIND', right_on  = 'IDIND')

del df_churn
del df_second

df['CHURN_DELAY'] = df['CHURN_DELAY'].apply(lambda x : x.days)
df['CHURN_DELAY_2'] = df['CHURN_DELAY_2'].apply(lambda x : x.days)

gc.collect()

0

### Recruitment departmenent

In [29]:
### Add DEPARTEMENT_CMD field
cat_order = df.groupby(['NUMCDE', 'DEPARTEMENT_NIV2']).agg({'DN': sum, 'DEPARTEMENT_NIV2': 'first'})
cat_order = cat_order.sort_values(by='DN',ascending=False).groupby('NUMCDE').agg({'DEPARTEMENT_NIV2': 'first'})
cat_order = cat_order.rename(columns = {'DEPARTEMENT_NIV2': 'DEPARTEMENT_CMD'})

df = pd.merge(df,cat_order, how='left', on='NUMCDE', right_index = False,  suffixes=(False,False))

del cat_order
gc.collect()

318

In [30]:
### Add DEPARTEMENT_FIRST (majority department of first command)
cat_rec = df.sort_values(by='DATCDE',ascending=True).drop_duplicates(subset='IDIND', keep='first', inplace=False)[['IDIND','DEPARTEMENT_CMD']]
cat_rec.columns = ['IDIND','DEPARTEMENT_FIRST']
df = pd.merge(df,cat_rec,on='IDIND',how='left',suffixes=(False,False))

### Recruitment month

In [34]:
df['MONTH_RECRUITMENT'] = pd.to_datetime(df['DATPREMCDEWEB']).dt.month

### Add SEGMENT column

In [20]:
df_seg = pd.read_csv('./Vertbaudet/France/src/SEGFR.CSV')
df = df.merge(df_seg.drop_duplicates(['IDIND', 'NUMCDE']), how = 'left', on = ['IDIND', 'NUMCDE'])

del df_seg

df = df.drop(columns = ['DATCDE_y', 'Unnamed: 0']).rename(columns = {'DATCDE_x': 'DATCDE'})

60

- Group VB segments to create Personna segments 'SEG_SMALL'

In [72]:
dict_segment = {'MP_GARCON': 'ENFANT', 'FM_MULTI': 'FM', 'FM_PRIMI': 'FM', 'MM_AUTRE': 'OTHER', 'BB': 'BB',
       'MP_FILLE': 'ENFANT', 'MP_MIXTE': 'ENFANT', 'ADOMP': 'ENFANT', 'MPBB': 'BB', 'ADO': 'ENFANT', 'ADOMPBB': 'BB', 'ADOBB': 'BB',
       'MM_KDO': 'KDO'}

def map_seg_small(x) : 
    if isinstance(x, str):
        return dict_segment[x]
    else:
        return np.nan

In [73]:
df['SEG_SMALL'] = df['SEGMENT'].apply(map_seg_small)

# Analysis

In [75]:
df.groupby(['SEG_SMALL', 'DEPARTEMENT_NIV2'])['DN'].sum().unstack().to_excel('./Vertbaudet/France/Output/'+today+'_dn_by_univ_by_segment.xlsx')

- Churn & LTV

In [76]:
df.drop_duplicates(['IDIND', 'SEG_SMALL']).groupby(['SEG_SMALL']).agg({'LTV_12': 'mean', 'LTV_24': 'mean', 'CHURN_DELAY': 'mean'}).to_excel('./Vertbaudet/France/Output/'+today+'_ltv_churn_by_univ_by_segment.xlsx')

- Recruit dept

In [83]:
df.drop_duplicates(['IDIND', 'SEG_SMALL']).groupby(['SEG_SMALL', 'DEPARTEMENT_FIRST']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_recruit_dept_by_univ_by_segment.xlsx')

- Client Category

In [81]:
df.drop_duplicates(['IDIND', 'SEG_SMALL']).groupby(['SEG_SMALL', 'CLIENT_CATEGORY']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_client_cat_by_segment.xlsx')

- Recruitment month

In [80]:
df.drop_duplicates(['IDIND', 'SEG_SMALL']).groupby(['SEG_SMALL', 'MONTH_RECRUITMENT']).agg({'IDIND': pd.Series.nunique}).unstack().to_excel('./Vertbaudet/France/Output/'+today+'_recruit_month_by_segment.xlsx')

- DN

In [86]:
df.groupby('SEG_SMALL')['DN'].sum().to_excel('./Vertbaudet/France/Output/'+today+'_dn_by_segment.xlsx')

- Average number of order

In [89]:
df.groupby('IDIND').agg({'NUMCDE': pd.Series.nunique, 'SEG_SMALL': 'first'}).groupby('SEG_SMALL').agg({'NUMCDE': 'mean'}).to_excel('./Vertbaudet/France/Output/'+today+'_avg_nb_order_by_segment.xlsx')

- Number of order

In [90]:
df.groupby('SEG_SMALL').agg({'NUMCDE': pd.Series.nunique}).to_excel('./Vertbaudet/France/Output/'+today+'_nb_order_by_segment.xlsx')